In [7]:
import pandas as pd
import os
import scanpy as sc
import numpy as np

from scGIST import scGIST
from scGIST import test_classifier, get_priority_score_list

In [ ]:
path = '/storage/adult_brain_genes/sketches_gene_choice_scripts'

In [2]:
adata = sc.read(os.path.join(path, 'combined_sketches.h5ad'))

In [14]:
uni = np.unique(adata.obs['Clusters'], return_counts=True)

In [1]:
adata = adata[adata.obs['Clusters'].isin(uni[0][np.where(uni[1] > 1)]), :]
adata.X = adata.X.todense()

In [2]:
for label_column in ['celltype', 'Clusters']:
    n_genes = adata.X.shape[1]
    n_classes = adata.obs[label_column].unique().size
    import numpy as np
    if label_column =='celltype':
        listed = [500, 1000]
    else:
        listed = [500, 1000]
    for panel_size in listed:
        pather =os.path.join(f'scGIST_{panel_size}_genes_{label_column}')
        try:
            os.mkdir(pather)
        except:
            None
        gist = scGIST()
        gist.create_model(n_genes, n_classes, panel_size=panel_size, alpha=0.2)
        gist.compile_model()
        
        gist.train_model(adata, label_column, verbose=2, epochs=200)
        # get marker names and plot the marker weights
        markers = gist.get_markers_names(adata, verbose=0, plot_weights=True)
        pd.DataFrame(markers).to_csv(os.path.join(pather, 'markers.csv'))

Run the model with priority given to genes of interest

In [9]:
# Load the data and convert it to list using utility function
gene_priorities = pd.read_csv('./data/head_neck/gene_priorities.csv')
priority_scores = get_priority_score_list(adata, gene_priorities)

In [12]:
panel_size = 350
gist = scGIST()
gist.create_model(n_genes, n_classes, panel_size=panel_size, priority_scores=priority_scores, alpha=0.2)
gist.compile_model()
gist.train_model(adata, label_column, verbose=0, epochs=200)

In [13]:
markers_indices = gist.get_markers_indices()
accuracy, f1 = test_classifier(adata, label_column, markers=markers_indices)
print(f'Accuracy: {accuracy}, F1-score: {f1}')

Accuracy: 90.14722536806342, F1-score: 0.8682288854464917
